<a href="https://colab.research.google.com/github/alinapradhan/-Multiple_Linear_Regression_PySpark/blob/main/multilinear_regression_pyspark_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
pip install pyspark

In [16]:
import pyspark
from pyspark.sql import SparkSession
#SparkSession is now the entry point of Spark
#SparkSession can also be construed as gateway to spark libraries

In [47]:
spark=SparkSession.builder.appName('cruise_ship_info').getOrCreate()

In [18]:
df=spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)

In [19]:
df.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)

In [20]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [21]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [22]:
#In our predictive model, below are the column
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [23]:
#columns identified as features are as below:
#['Cruise_line','Age','Tonnage','passengers','length','cabins','passenger_density']
#to work on the features, spark MLlib expects every value to be in numeric form
#feature 'Cruise_line is string datatype
#using StringIndexer, string type will be typecast to numeric datatype
#import library strinindexer for typecasting

from pyspark.ml.feature import StringIndexer
indexer=StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')
indexed=indexer.fit(df).transform(df)
indexed.head(5)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0),
 Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)]

In [24]:
#above code will convert string to numeric feature and create a new dataframe
#new dataframe contains a new feature 'cruise_cat' and can be used further
#feature cruise_cat is now vectorized and can be used to fed to model
for item in indexed.head(5):
    print(item)
    print('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)




In [26]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [29]:
assembler=VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density','cruise_cat'],outputCol='features')
output=assembler.transform(indexed)
output.select('features','crew').show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [30]:
final_data=output.select('features','crew')

In [31]:
train_data,test_data=final_data.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              114|
|   mean|7.759649122807029|
| stddev|3.611308426091859|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [33]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               44|
|   mean|7.883636363636366|
| stddev|3.245392505669575|
|    min|             0.59|
|    max|             13.0|
+-------+-----------------+



In [34]:
from pyspark.ml.regression import LinearRegression
ship_lr=LinearRegression(labelCol='crew')

In [35]:
trained_ship_model=ship_lr.fit(train_data)

In [45]:
ship_results_train=trained_ship_model.evaluate(train_data)

In [46]:
print('Rsquared Error :',ship_results_train.r2)

Rsquared Error : 0.9311190883306063


In [44]:
ship_results_train

In [38]:
unlabeled_data=test_data.select('features')

In [39]:
unlabeled_data.show(5)

+--------------------+
|            features|
+--------------------+
|[5.0,86.0,21.04,9...|
|[5.0,122.0,28.5,1...|
|[6.0,93.0,23.94,9...|
|[7.0,89.6,25.5,9....|
|[8.0,77.499,19.5,...|
+--------------------+
only showing top 5 rows



In [40]:
predictions=trained_ship_model.transform(unlabeled_data)

In [41]:

#testing Model on unlabeled data
#create unlabeled data from test_data
#testing model on unlabeled data
predictions.show()#below are the results of output from test data


+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,86.0,21.04,9...| 9.229487557427635|
|[5.0,122.0,28.5,1...| 5.676668990623469|
|[6.0,93.0,23.94,9...|10.615418861282597|
|[7.0,89.6,25.5,9....| 11.25577796168969|
|[8.0,77.499,19.5,...|  8.59769023048616|
|[9.0,88.5,21.24,9...| 9.532101705066196|
|[9.0,90.09,25.01,...| 9.248314205411852|
|[9.0,105.0,27.2,8...|11.373962418498564|
|[9.0,110.0,29.74,...|12.268488510446547|
|[9.0,116.0,26.0,9...|11.110488968325036|
|[10.0,46.0,7.0,6....| 2.602350463400204|
|[10.0,77.0,20.16,...|  8.81143066126804|
|[10.0,86.0,21.14,...| 9.683511234721491|
|[10.0,90.09,25.01...| 8.829855094036555|
|[10.0,91.62700000...| 9.094277420087776|
|[10.0,105.0,27.2,...|11.363681798330223|
|[10.0,110.0,29.74...| 12.25424233502823|
|[11.0,91.0,20.32,...| 9.113465686142028|
|[11.0,91.62700000...| 9.083996799919435|
|[11.0,108.977,26....|11.089929447223835|
+--------------------+------------